# Feature Engineering for Stock Price Prediction

This notebook creates engineered features including:
- Technical indicators (SMA, EMA, RSI, MACD, Bollinger Bands)
- Lag features and rolling statistics
- Volatility features
- Target variable definitions

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import argrelextrema
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
plt.rcParams['figure.figsize'] = (14, 6)

## 1. Load Data

In [ ]:
# Load cleaned data from EDA
df = pd.read_csv('../stock_data.csv', parse_dates=['dt'])
df.set_index('dt', inplace=True)

print(f"Dataset shape: {df.shape}")
print(f"Date range: {df.index.min()} to {df.index.max()}")
df.head()

## 2. Price-Based Features

In [ ]:
# Daily returns
df['daily_return'] = df['sp500'].pct_change()

# Log returns
df['log_return'] = np.log(df['sp500'] / df['sp500'].shift(1))

# Price change
df['price_change'] = df['sp500'].diff()

# Percentage change over multiple periods
for period in [3, 5, 10, 20]:
    df[f'pct_change_{period}d'] = df['sp500'].pct_change(periods=period)
    df[f'price_change_{period}d'] = df['sp500'].diff(periods=period)

print("Price-based features created:")
print([col for col in df.columns if 'return' in col or 'change' in col])

## 3. Technical Indicators

In [ ]:
# Simple Moving Averages
for window in [5, 10, 20, 50, 200]:
    df[f'SMA_{window}'] = df['sp500'].rolling(window=window).mean()
    df[f'price_to_SMA_{window}'] = df['sp500'] / df[f'SMA_{window}']

# Exponential Moving Averages
for span in [12, 26]:
    df[f'EMA_{span}'] = df['sp500'].ewm(span=span, adjust=False).mean()

# MACD (Moving Average Convergence Divergence)
df['MACD'] = df['EMA_12'] - df['EMA_26']
df['MACD_signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
df['MACD_histogram'] = df['MACD'] - df['MACD_signal']

print("\nMoving averages and MACD created")

In [ ]:
# RSI (Relative Strength Index)
def calculate_rsi(prices, period=14):
    delta = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

df['RSI_14'] = calculate_rsi(df['sp500'], period=14)

print("RSI calculated")

In [ ]:
# Bollinger Bands
window = 20
df['BB_middle'] = df['sp500'].rolling(window=window).mean()
df['BB_std'] = df['sp500'].rolling(window=window).std()
df['BB_upper'] = df['BB_middle'] + (2 * df['BB_std'])
df['BB_lower'] = df['BB_middle'] - (2 * df['BB_std'])
df['BB_width'] = df['BB_upper'] - df['BB_lower']
df['BB_position'] = (df['sp500'] - df['BB_lower']) / (df['BB_upper'] - df['BB_lower'])

print("Bollinger Bands calculated")

In [ ]:
# Visualize technical indicators
fig, axes = plt.subplots(3, 1, figsize=(16, 12))

# Price with moving averages
axes[0].plot(df.index[-500:], df['sp500'][-500:], label='S&P 500', linewidth=1.5)
axes[0].plot(df.index[-500:], df['SMA_20'][-500:], label='SMA 20', linewidth=1)
axes[0].plot(df.index[-500:], df['SMA_50'][-500:], label='SMA 50', linewidth=1)
axes[0].set_title('S&P 500 with Moving Averages (Last 500 Days)', fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# RSI
axes[1].plot(df.index[-500:], df['RSI_14'][-500:], linewidth=1, color='purple')
axes[1].axhline(70, color='red', linestyle='--', alpha=0.7, label='Overbought')
axes[1].axhline(30, color='green', linestyle='--', alpha=0.7, label='Oversold')
axes[1].set_title('RSI (14-day)', fontweight='bold')
axes[1].set_ylim(0, 100)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# MACD
axes[2].plot(df.index[-500:], df['MACD'][-500:], label='MACD', linewidth=1)
axes[2].plot(df.index[-500:], df['MACD_signal'][-500:], label='Signal', linewidth=1)
axes[2].bar(df.index[-500:], df['MACD_histogram'][-500:], label='Histogram', alpha=0.3)
axes[2].set_title('MACD', fontweight='bold')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Lag Features

In [ ]:
# Lagged price features
for lag in [1, 2, 3, 5, 10]:
    df[f'sp500_lag_{lag}'] = df['sp500'].shift(lag)
    df[f'return_lag_{lag}'] = df['daily_return'].shift(lag)
    df[f'vix_lag_{lag}'] = df['vix'].shift(lag)

# Lagged volume
for lag in [1, 2, 5]:
    df[f'volume_lag_{lag}'] = df['sp500_volume'].shift(lag)

print(f"Lag features created: {len([col for col in df.columns if 'lag' in col])} features")

## 5. Rolling Window Features

In [ ]:
# Rolling statistics
for window in [5, 10, 20]:
    df[f'rolling_mean_{window}'] = df['sp500'].rolling(window=window).mean()
    df[f'rolling_std_{window}'] = df['sp500'].rolling(window=window).std()
    df[f'rolling_min_{window}'] = df['sp500'].rolling(window=window).min()
    df[f'rolling_max_{window}'] = df['sp500'].rolling(window=window).max()
    df[f'rolling_median_{window}'] = df['sp500'].rolling(window=window).median()
    
    # Position relative to rolling window
    df[f'position_in_range_{window}'] = (df['sp500'] - df[f'rolling_min_{window}']) / \
                                         (df[f'rolling_max_{window}'] - df[f'rolling_min_{window}'])

print(f"Rolling window features created")

## 6. Volatility Features

In [ ]:
# Historical volatility (rolling std of returns)
for window in [5, 10, 20]:
    df[f'volatility_{window}'] = df['daily_return'].rolling(window=window).std() * np.sqrt(252)

# VIX changes
df['vix_change'] = df['vix'].diff()
df['vix_pct_change'] = df['vix'].pct_change()
df['vix_momentum'] = df['vix'].diff(5)

# Volatility ratio (short-term vs long-term)
df['volatility_ratio'] = df['volatility_5'] / df['volatility_20']

print("Volatility features created")

## 7. Volume Features

In [ ]:
# Volume ratios
df['volume_ma_20'] = df['sp500_volume'].rolling(window=20).mean()
df['volume_ratio'] = df['sp500_volume'] / df['volume_ma_20']
df['volume_momentum'] = df['sp500_volume'].diff(5)
df['volume_change'] = df['sp500_volume'].pct_change()

print("Volume features created")

## 8. Temporal Features

In [ ]:
# Extract temporal features
df['day_of_week'] = df.index.dayofweek
df['month'] = df.index.month
df['quarter'] = df.index.quarter
df['day_of_month'] = df.index.day
df['week_of_year'] = df.index.isocalendar().week
df['is_month_end'] = df.index.is_month_end.astype(int)
df['is_quarter_end'] = df.index.is_quarter_end.astype(int)

print("Temporal features created")

## 9. Target Variable Engineering

We'll create multiple target variables for different prediction tasks

In [ ]:
# REGRESSION TARGETS

# Next day price
df['target_price_next_day'] = df['sp500'].shift(-1)

# 5-day ahead price
df['target_price_5day'] = df['sp500'].shift(-5)

# Next day return
df['target_return_next_day'] = df['daily_return'].shift(-1)

print("Regression targets created")

In [ ]:
# CLASSIFICATION TARGETS

# Binary direction (next day)
df['target_direction_next_day'] = (df['sp500'].shift(-1) > df['sp500']).astype(int)

# Binary direction (5-day)
df['target_direction_5day'] = (df['sp500'].shift(-5) > df['sp500']).astype(int)

# Multi-class direction (-1: down, 0: flat, 1: up)
threshold = 0.005  # 0.5% threshold for "flat"
next_return = df['daily_return'].shift(-1)
df['target_multiclass'] = 0
df.loc[next_return > threshold, 'target_multiclass'] = 1
df.loc[next_return < -threshold, 'target_multiclass'] = -1

print(f"\nDirection distribution:")
print(df['target_direction_next_day'].value_counts())
print(f"\nMulti-class distribution:")
print(df['target_multiclass'].value_counts())

In [ ]:
# BUY/SELL/HOLD SIGNALS based on local extrema
def identify_signals(prices, order=5):
    """
    Identify buy/sell signals based on local minima/maxima
    order: how many points on each side to compare
    """
    signals = np.zeros(len(prices))
    
    # Find local minima (BUY signals)
    local_min = argrelextrema(prices.values, np.less_equal, order=order)[0]
    signals[local_min] = 1  # BUY
    
    # Find local maxima (SELL signals)
    local_max = argrelextrema(prices.values, np.greater_equal, order=order)[0]
    signals[local_max] = -1  # SELL
    
    return signals

df['target_signals'] = identify_signals(df['sp500'], order=10)

print(f"\nSignals distribution:")
print(f"BUY signals: {(df['target_signals'] == 1).sum()}")
print(f"SELL signals: {(df['target_signals'] == -1).sum()}")
print(f"HOLD: {(df['target_signals'] == 0).sum()}")

In [ ]:
# Visualize signals
fig, ax = plt.subplots(figsize=(16, 6))

# Plot recent data
recent = df.tail(1000)
ax.plot(recent.index, recent['sp500'], label='S&P 500', linewidth=1.5, color='blue', alpha=0.7)

# Plot BUY signals
buy_signals = recent[recent['target_signals'] == 1]
ax.scatter(buy_signals.index, buy_signals['sp500'], color='green', marker='^', 
           s=100, label='BUY Signals', zorder=5)

# Plot SELL signals
sell_signals = recent[recent['target_signals'] == -1]
ax.scatter(sell_signals.index, sell_signals['sp500'], color='red', marker='v', 
           s=100, label='SELL Signals', zorder=5)

ax.set_title('BUY/SELL Signals Based on Local Extrema (Last 1000 Days)', fontweight='bold', fontsize=14)
ax.set_xlabel('Date')
ax.set_ylabel('S&P 500 Price')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 10. Feature Summary and Data Quality

In [ ]:
print("="*80)
print("FEATURE ENGINEERING SUMMARY")
print("="*80)

print(f"\nTotal features created: {len(df.columns)}")
print(f"Date range: {df.index.min()} to {df.index.max()}")
print(f"Total observations: {len(df):,}")

# Count features by category
categories = {
    'Price-based': ['return', 'change'],
    'Technical Indicators': ['SMA', 'EMA', 'MACD', 'RSI', 'BB'],
    'Lag Features': ['lag'],
    'Rolling Features': ['rolling'],
    'Volatility': ['volatility', 'vix'],
    'Volume': ['volume'],
    'Temporal': ['day', 'month', 'quarter', 'week'],
    'Target Variables': ['target']
}

for category, keywords in categories.items():
    count = sum(1 for col in df.columns if any(kw in col.lower() for kw in keywords))
    print(f"{category}: {count} features")

# Missing values after feature engineering
missing = df.isnull().sum()
print(f"\nFeatures with missing values: {(missing > 0).sum()}")
print(f"Max missing count: {missing.max()} ({missing.max()/len(df)*100:.2f}%)")

## 11. Handle Missing Values and Save

In [ ]:
# Drop rows with missing values (mostly from rolling windows at start)
print(f"Rows before cleaning: {len(df):,}")
df_clean = df.dropna()
print(f"Rows after cleaning: {len(df_clean):,}")
print(f"Rows removed: {len(df) - len(df_clean):,}")

# Alternative: could use forward fill for specific features
# df_clean = df.fillna(method='ffill').dropna()

In [ ]:
# Save engineered features
output_path = '../data/processed/features_engineered.csv'
df_clean.to_csv(output_path)
print(f"\nEngineered features saved to: {output_path}")
print(f"Final dataset shape: {df_clean.shape}")
print(f"Features: {df_clean.shape[1]} columns")
print(f"Date range: {df_clean.index.min()} to {df_clean.index.max()}")

In [ ]:
# Display sample of engineered features
feature_cols = [col for col in df_clean.columns if 'target' not in col]
print(f"\nSample of engineered features (first 5 rows, random 10 features):")
sample_features = np.random.choice(feature_cols, size=min(10, len(feature_cols)), replace=False)
df_clean[sample_features].head()

## 12. Feature Correlation with Targets

In [ ]:
# Calculate correlation with regression target
target_col = 'target_return_next_day'
correlations = df_clean.corr()[target_col].sort_values(ascending=False)

print(f"\nTop 20 features correlated with {target_col}:")
print(correlations.head(20))

print(f"\nBottom 20 features (negative correlation):")
print(correlations.tail(20))

In [ ]:
# Visualize top correlations
top_features = correlations.head(15).index.tolist()
top_features = [f for f in top_features if f != target_col]

plt.figure(figsize=(10, 8))
sns.barplot(x=correlations[top_features].values, y=top_features, palette='viridis')
plt.title(f'Top Features Correlated with {target_col}', fontweight='bold', fontsize=13)
plt.xlabel('Correlation Coefficient')
plt.tight_layout()
plt.show()

## Next Steps

With engineered features ready, you can now:
1. Split data temporally (use `src/preprocessing.py`)
2. Train baseline models for benchmarking
3. Train XGBoost/LightGBM models
4. Train LSTM/GRU deep learning models
5. Evaluate and compare models
6. Backtest trading strategies